In [234]:
import os, shutil, csv, math
import pandas as pd
import numpy as np
from scipy import signal
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt


window_size = 250  # Window interval
lowcut = 20        # Lower cutoff frequency in Hz
highcut = 450      # Upper cutoff frequency in Hz
fs = 2000          # Sampling frequency in Hz

epochsize = window_size*fs/1000
stride = 10
no_of_grids = 2
no_of_channels = 32


def create_or_empty_directory(directory_path):

    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    
    else:
        
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)



def rms_calc(signal, windowlength, overlap, zeropad):
    
    delta = windowlength - overlap
    
    # Calculate RMS
    indices = np.arange(0, len(signal), delta)
    
    if len(signal) - indices[-1] + 1 < windowlength:
        if zeropad:
            zero_arr = np.zeros(int(windowlength - 1))
            signal = np.concatenate((zero_arr, signal))  # Zero padding to the start
        else:
            indices = indices[:np.argmax(indices + windowlength - 1 <= len(signal))]
    
    y = np.zeros(len(indices))
    signal_squared = signal ** 2
    
    index = 0
    for i in indices:
        y[index] = np.sqrt(np.mean(signal_squared[i:i+windowlength]))
        index += 1
        
    y = [1000*x for x in y]
        
    return y



def arv_calc(signal, windowlength, overlap, zeropad):
    
    delta = windowlength - overlap
    
    # Calculate ARV
    indices = np.arange(0, len(signal), delta)
    
    if len(signal) - indices[-1] + 1 < windowlength:
        if zeropad:
            zero_arr = np.zeros(windowlength - 1)
            signal = np.concatenate((zero_arr, signal))
        else:
            indices = indices[:np.argmax(indices + windowlength - 1 <= len(signal))]
    
    y = np.zeros(len(indices))
    
    index = 0
    for i in indices:
        y[index] = np.mean(np.abs(signal[i:i+windowlength]))
        index += 1
                
    y = [1000*x for x in y]
        
    return y


def downsample_list(lst,length_req):
        
    print(lst)
    print(length_req)
    
    for i in range(length_req):
        
        if(len(lst)>10*i):
            downsampled_list.append(float(lst[10*i]))
        else:
            downsampled_list.append(0)
            
    return downsampled_list
    
    


In [235]:
# Create folder if it doesn't exist or empty folder if it exists

create_or_empty_directory("EMG_RMS_Data")
create_or_empty_directory("EMG_ARV_Data")

# Set the folder paths

Input_folder = "Rectified_EMG_Data"
RMS_output_folder = "EMG_RMS_Data"
ARV_output_folder = "EMG_ARV_Data"

# Get a list of CSV files in the input folder

csv_files = [file for file in os.listdir(Input_folder) if file.endswith(".csv")]
csv_files.sort()

print(csv_files)

['P01_Left_120.csv', 'P01_Left_90.csv', 'P01_Left_Extension.csv', 'P01_Left_Flexion.csv', 'P01_Left_Pull1.csv', 'P01_Left_Push3.csv', 'P01_Right_120.csv', 'P01_Right_60.csv', 'P01_Right_90.csv', 'P01_Right_Extension.csv', 'P01_Right_Flexion.csv', 'P01_Right_Pull2.csv', 'P01_Right_Push1.csv', 'P02_Left_120.csv', 'P02_Left_60.csv', 'P02_Left_90.csv', 'P02_Left_Extension3.csv', 'P02_Left_Flexion2.csv', 'P02_Left_Pull3.csv', 'P02_Left_Push2.csv', 'P02_Right_120.csv', 'P02_Right_60.csv', 'P02_Right_90.csv', 'P02_Right_Extension3.csv', 'P02_Right_Flexion3.csv', 'P02_Right_Pull3.csv', 'P02_Right_Push1.csv', 'P03_Left_120.csv', 'P03_Left_60.csv', 'P03_Left_90.csv', 'P03_Left_Extension.csv', 'P03_Left_Flexion.csv', 'P03_Left_Pull.csv', 'P03_Left_Push3.csv', 'P03_Right_120.csv', 'P03_Right_60.csv', 'P03_Right_90.csv', 'P03_Right_Extension2.csv', 'P03_Right_Flexion1.csv', 'P03_Right_Pull.csv', 'P03_Right_Push2.csv', 'P04_Left_120.csv', 'P04_Left_60.csv', 'P04_Left_90.csv', 'P04_Left_Extension.csv

In [236]:
for csv_file in csv_files:
    
    print(csv_file)
    
    input_path = os.path.join(Input_folder, csv_file)
    df = pd.read_csv(input_path, header = None)
    
    if(df.shape[1] == no_of_grids*no_of_channels+2):
        torque_data = df.iloc[:, -1].tolist()
        print(len(torque_data))
    
    
    # -------- Bandpass Filtering and RMS, ARV Processing ------- #
    
    
    columns_to_filter_bandpass = df.columns[1:no_of_grids*no_of_channels+1]  # Exclude the first and the 66th column
    
    order = 2
    lowcut = 20
    highcut = 450
    fs = 2000
    
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')

    rms_columns = {}
    arv_columns = {}
    
    for col in columns_to_filter_bandpass:
        
        signal_data = df[col].to_numpy()
        filtered_signal = lfilter(b, a, signal_data)
        processed_signal_rms = rms_calc(filtered_signal,int(epochsize),int(epochsize-stride),1)
        rms_columns[col] = processed_signal_rms
        processed_signal_arv = arv_calc(filtered_signal,int(epochsize),int(epochsize-stride),1)
        arv_columns[col] = processed_signal_arv

    rms_df = pd.DataFrame(rms_columns)
    arv_df = pd.DataFrame(arv_columns)
    
    if(df.shape[1] == no_of_grids*no_of_channels+2):
        
        #torque_downsampled = downsample_list(torque_data, rms_df.shape[0])
        torque_df = pd.DataFrame({"Torque": torque_data[::10]})
        rms_df = pd.concat([rms_df, torque_df], axis=1)
        arv_df = pd.concat([arv_df, torque_df], axis=1)
    
    # Concatenate the original DataFrame and the new DataFrame
    
    print(rms_df)

    # Save the modified DataFrame to a new CSV file
    output_path_rms = os.path.join(RMS_output_folder, csv_file)
    output_path_arv = os.path.join(ARV_output_folder, csv_file)
    
    custom_index_values = [0.0005 * i for i in range(len(rms_df))]  # Adjust the range accordingly
    rms_df.index = custom_index_values
    arv_df.index = custom_index_values
    rms_df.to_csv(output_path_rms, header = False, index = True)
    arv_df.to_csv(output_path_arv, header = False, index = True)


P01_Left_120.csv
27000
             1         2         3         4         5         6         7  \
0     0.044393  0.062151  0.076949  0.076949  0.088787  0.091746  0.082868   
1     1.548959  1.586953  1.507540  1.476394  1.444810  1.511808  1.398784   
2     2.102032  2.220838  1.995527  1.964300  2.144147  2.272301  1.959351   
3     2.529179  2.589992  2.335642  2.376905  2.518646  2.692726  2.274102   
4     2.971466  3.024953  2.814915  2.985031  2.968372  3.046156  2.687768   
...        ...       ...       ...       ...       ...       ...       ...   
2695  7.744796  7.455679  7.675602  7.626042  7.615173  7.778737  7.687491   
2696  7.736340  7.499066  7.694470  7.635527  7.650665  7.788759  7.699591   
2697  7.685120  7.509809  7.691758  7.623347  7.626063  7.818426  7.686949   
2698  7.775458  7.597954  7.754224  7.701352  7.724956  7.893446  7.706771   
2699  7.777505  7.576249  7.738367  7.689562  7.734466  7.887957  7.708030   

             8         9        10  ... 

P01_Left_Flexion.csv
30625
              1          2          3          4          5          6  \
0      0.251562   0.281158   0.263401   0.281158   0.263401   0.275239   
1      1.763515   1.981995   1.954789   1.950923   1.802224   2.164642   
2      2.588967   2.562386   2.597098   2.668211   2.538096   2.817766   
3      3.460690   3.617075   3.592059   3.742802   3.616158   3.991017   
4      4.571109   4.615581   4.664558   4.998078   4.909485   5.066040   
...         ...        ...        ...        ...        ...        ...   
3058  23.665116  23.872165  23.849094  24.008119  23.984390  23.985257   
3059  24.291858  24.445181  24.474231  24.585921  24.557728  24.586513   
3060  26.182260  26.277290  26.238658  26.485745  26.407393  26.421100   
3061  27.508533  27.386513  27.442248  27.602854  27.513146  27.509267   
3062  27.477133  27.376021  27.413263  27.590654  27.453733  27.451474   

              7          8          9         10  ...         56         57  \
0    

32500
               1           2           3           4           5           6  \
0       0.526801    0.665901    0.843474    0.976654    1.118713    1.169026   
1       6.189269    6.686892    7.358978    8.860386   10.255738   11.096767   
2      12.058306   12.683109   13.856116   16.247730   17.752951   18.204209   
3      19.554868   21.655057   24.531025   27.922044   30.442731   31.215357   
4      21.181530   23.734593   27.143719   30.836841   33.820364   34.728366   
...          ...         ...         ...         ...         ...         ...   
3245  171.836290  178.378234  181.728692  184.043336  186.438059  187.825902   
3246  176.878711  183.690125  186.753936  188.756089  190.398398  190.874877   
3247  177.088263  184.224559  187.290655  189.113908  190.454754  190.784509   
3248  177.773769  185.565527  188.844932  190.473741  191.364074  191.409955   
3249  172.281001  180.299412  183.671510  185.257337  186.210915  186.234720   

               7           8     

P01_Right_Extension.csv
29625
              1          2          3          4          5          6  \
0      0.307794   0.319632   0.361066   0.343309   0.405460   0.396581   
1      5.379953   5.789913   5.925774   5.713071   6.081135   6.189308   
2     12.497309  12.900678  13.045375  13.103599  13.738684  13.784956   
3     14.540583  14.996170  15.136009  15.479378  16.323469  16.505908   
4     16.217168  16.676818  16.786714  17.099224  17.948675  18.180243   
...         ...        ...        ...        ...        ...        ...   
2958  18.584984  18.622850  18.959649  18.846406  18.822683  19.188453   
2959  18.615507  18.674787  19.004502  18.893758  18.866284  19.233247   
2960  18.578893  18.675955  18.997495  18.836307  18.809294  19.272309   
2961  18.605211  18.697192  19.039594  18.859978  18.849871  19.284348   
2962  18.613379  18.720378  19.059965  18.890280  18.884882  19.299606   

              7          8          9         10  ...         56         57  \
0 

            1          2          3          4          5           6   \
0     0.284119   0.251566   0.284119   0.284119   0.340352    0.245638   
1     2.495933   2.285308   2.246258   2.557554   2.851803    2.296118   
2     5.310494   5.190401   5.177309   5.320076   5.611983    5.067602   
3     6.520458   6.483779   6.422660   6.566998   6.726504    6.404621   
4     6.595156   6.557918   6.551322   6.650157   6.840511    6.511214   
..         ...        ...        ...        ...        ...         ...   
795  50.314843  58.028276  68.365360  80.208267  91.677315   97.600220   
796  50.120576  58.007272  68.265346  79.940979  91.161183   96.725584   
797  50.753760  58.525139  68.987304  80.667881  91.881912   97.618311   
798  52.745511  61.151715  72.195908  83.985708  95.518496  101.331988   
799  52.223167  60.332529  71.193738  82.738983  94.027013   99.575814   

             7           8          9          10  ...         55         56  \
0      0.301870    0.298912   0

...

            1         2         3         4         5         6         7  \
0    0.544530  0.446878  0.426190  0.538633  0.689558  0.763521  0.822794   
1    4.671222  4.944239  4.404918  5.025353  6.022313  6.079611  6.400272   
2    4.957685  5.378492  4.767252  5.365465  6.362705  6.402422  6.742858   
3    5.135342  5.923372  5.489419  6.244212  7.409026  7.489864  7.931722   
4    6.857528  6.610713  6.100057  6.727769  7.884942  7.937486  8.417597   
..        ...       ...       ...       ...       ...       ...       ...   
820  5.997681  4.890620  4.249721  4.408168  3.958466  4.117883  3.874912   
821  5.931634  4.804677  4.203553  4.400613  3.921338  4.108041  3.901073   
822  5.135105  4.405093  3.964398  4.162411  3.824042  3.997764  3.863318   
823  5.004756  4.203750  3.875449  4.057919  3.734444  3.900769  3.778511   
824  5.004810  4.187410  3.861749  4.063027  3.730070  3.895838  3.786792   

            8         9        10  ...        56        57        58  \
0  

P38_Left_90.csv
30000
               1           2           3           4           5           6  \
0       7.102987    7.070402    7.111780    7.070402    7.082195    7.055610   
1      51.566964   51.478510   51.711820   51.464411   51.454986   51.576539   
2      91.153985   90.849723   91.198370   91.100940   90.981413   90.947513   
3     115.262042  114.897774  115.280572  115.268386  114.909699  114.854868   
4     133.517992  133.274029  133.627593  133.539979  133.250997  133.263556   
...          ...         ...         ...         ...         ...         ...   
2995  507.919189  508.744768  509.129952  508.788210  508.230574  507.945927   
2996  507.780924  508.611786  508.989231  508.622451  508.029527  507.733437   
2997  508.108985  508.893356  509.295280  508.914935  508.317897  507.984041   
2998  508.019463  508.834598  509.254768  508.844214  508.306146  507.969986   
2999  508.127813  508.933677  509.358532  508.949018  508.405308  508.040488   

               7 

P38_Left_Pull2.csv
8750
             1          2          3          4          5          6  \
0     0.026585   0.056274   0.014793   0.008896   0.005896   0.020689   
1     0.654198   0.776569   0.676371   0.529521   0.659801   0.532264   
2     0.693683   0.931990   0.761455   0.668786   0.737062   0.622381   
3     0.817102   1.042923   0.902434   0.759148   0.875042   0.730030   
4     1.178045   1.246920   1.159950   1.021415   1.289211   1.025772   
..         ...        ...        ...        ...        ...        ...   
870  60.069779  60.326473  61.195881  61.608311  62.212947  62.900938   
871  62.092141  62.276824  63.045886  63.298288  63.416704  63.916817   
872  64.727581  65.152533  66.322154  66.653490  66.641626  66.826981   
873  65.321694  65.718016  66.984032  67.412769  67.419845  67.596939   
874  66.088748  66.514492  67.812125  68.051669  67.853806  67.930317   

             7          8          9         10  ...         56          57  \
0     0.029585   0.0

P38_Right_60.csv
41875
               1           2           3           4           5           6  \
0       2.184118    2.133844    2.089467    2.104259    2.095363    2.095363   
1      34.220812   34.374199   34.512215   34.836854   34.771559   35.076960   
2      43.875387   44.213001   44.374813   44.934300   44.856279   45.063066   
3      73.397859   73.807996   74.408043   75.506836   75.195175   75.605066   
4      81.748671   82.214780   82.844474   84.002598   83.798609   84.175130   
...          ...         ...         ...         ...         ...         ...   
4183  322.013013  323.380419  324.503933  325.220391  325.657041  326.055070   
4184  321.508444  322.787975  324.032835  324.708206  325.144363  325.534324   
4185  321.212935  322.498876  323.778156  324.421624  324.859541  325.267973   
4186  320.446023  321.679703  323.046992  323.683716  324.159453  324.543571   
4187  320.107616  321.350746  322.760520  323.375036  323.852681  324.203333   

               7

P38_Right_Flexion2.csv
28500
               1           2           3           4           5           6  \
0       4.087179    4.190726    4.211415    4.258792    4.247000    4.235104   
1      41.798627   42.193636   42.667340   43.011291   43.182332   43.151168   
2      68.247047   69.303521   70.036324   70.510306   70.953580   70.938264   
3      79.823112   80.413064   81.286446   81.863185   82.287116   82.139770   
4     100.557476  101.066537  101.842613  102.413903  102.946927  102.774748   
...          ...         ...         ...         ...         ...         ...   
2845  418.655251  421.660230  423.985843  425.118356  425.131295  425.063911   
2846  417.939632  420.911513  423.252160  424.329564  424.331980  424.283435   
2847  417.677686  420.728020  423.042908  424.134129  424.126435  424.043874   
2848  416.003897  419.018463  421.401575  422.458208  422.494198  422.369564   
2849  416.629572  419.613833  422.037061  423.095992  423.149567  423.006158   

          